In [48]:
# pip install sentence_transformers

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

In [2]:
df = pd.read_csv(r"Bank_Transaction_Fraud_Detection.csv")

In [3]:
df.head()

,Customer_ID,Customer_Name,Gender,Age,State,City,Bank_Branch,Account_Type,Transaction_ID,Transaction_Date,...,Merchant_Category,Account_Balance,Transaction_Device,Transaction_Location,Device_Type,Is_Fraud,Transaction_Currency,Customer_Contact,Transaction_Description,Customer_Email
0,d5f6ec07-d69e-4f47-b9b4-7c58ff17c19e,Osha Tella,Male,60,Kerala,Thiruvananthapuram,Thiruvananthapuram Branch,Savings,4fa3208f-9e23-42dc-b330-844829d0c12c,23-01-2025,...,Restaurant,74557.27,Voice Assistant,"Thiruvananthapuram, Kerala",POS,0,INR,+9198579XXXXXX,Bitcoin transaction,oshaXXXXX@XXXXX.com
1,7c14ad51-781a-4db9-b7bd-67439c175262,Hredhaan Khosla,Female,51,Maharashtra,Nashik,Nashik Branch,Business,c9de0c06-2c4c-40a9-97ed-3c7b8f97c79c,11-01-2025,...,Restaurant,74622.66,POS Mobile Device,"Nashik, Maharashtra",Desktop,0,INR,+9191074XXXXXX,Grocery delivery,hredhaanXXXX@XXXXXX.com
2,3a73a0e5-d4da-45aa-85f3-528413900a35,Ekani Nazareth,Male,20,Bihar,Bhagalpur,Bhagalpur Branch,Savings,e41c55f9-c016-4ff3-872b-cae72467c75c,25-01-2025,...,Groceries,66817.99,ATM,"Bhagalpur, Bihar",Desktop,0,INR,+9197745XXXXXX,Mutual fund investment,ekaniXXX@XXXXXX.com
3,7902f4ef-9050-4a79-857d-9c2ea3181940,Yamini Ramachandran,Female,57,Tamil Nadu,Chennai,Chennai Branch,Business,7f7ee11b-ff2c-45a3-802a-49bc47c02ecb,19-01-2025,...,Entertainment,58177.08,POS Mobile App,"Chennai, Tamil Nadu",Mobile,0,INR,+9195889XXXXXX,Food delivery,yaminiXXXXX@XXXXXXX.com
4,3a4bba70-d9a9-4c5f-8b92-1735fd8c19e9,Kritika Rege,Female,43,Punjab,Amritsar,Amritsar Branch,Savings,f8e6ac6f-81a1-4985-bf12-f60967d852ef,30-01-2025,...,Entertainment,16108.56,Virtual Card,"Amritsar, Punjab",Mobile,0,INR,+9195316XXXXXX,Debt repayment,kritikaXXXX@XXXXXX.com


In [9]:
df.columns

Index(['Customer_ID', 'Customer_Name', 'Gender', 'Age', 'State', 'City',
       'Bank_Branch', 'Account_Type', 'Transaction_ID', 'Transaction_Date',
       'Transaction_Time', 'Transaction_Amount', 'Merchant_ID',
       'Transaction_Type', 'Merchant_Category', 'Account_Balance',
       'Transaction_Device', 'Transaction_Location', 'Device_Type', 'Is_Fraud',
       'Transaction_Currency', 'Customer_Contact', 'Transaction_Description',
       'Customer_Email'],
      dtype='object')

In [19]:
# value_counts = df["Transaction_Description"].value_counts()
# value_counts[value_counts<=1071]

In [20]:
df["Transaction_Description"].value_counts()

Sports ticket                1268
Home appliance repair        1257
Taxi fare                    1248
Seminar registration         1246
Taxi booking                 1240
                             ... 
Food subscription            1098
Loyalty points redemption    1093
Penalty fee                  1092
Online subscription          1080
Medical treatment payment    1071
Name: Transaction_Description, Length: 172, dtype: int64

In [21]:
df["Is_Fraud"].value_counts()

0    189912
1     10088
Name: Is_Fraud, dtype: int64

In [23]:
df["Is_Fraud"].value_counts().to_frame("count").assign(percentage=lambda x: x["count"] / x["count"].sum() * 100)

,count,percentage
0,189912,94.956
1,10088,5.044


In [24]:
df[df["Is_Fraud"]==1].head()

,Customer_ID,Customer_Name,Gender,Age,State,City,Bank_Branch,Account_Type,Transaction_ID,Transaction_Date,...,Merchant_Category,Account_Balance,Transaction_Device,Transaction_Location,Device_Type,Is_Fraud,Transaction_Currency,Customer_Contact,Transaction_Description,Customer_Email
17,99d95f63-a2fa-4707-b577-f4a008e6700e,Theodore Suri,Male,56,Dadra and Nagar Haveli and Daman and Diu,Silvassa,Silvassa Branch,Business,aa3832eb-c0f0-48b1-b195-a376a1e17797,24-01-2025,...,Restaurant,74322.85,Biometric Scanner,"Silvassa, Dadra and Nagar Haveli and Daman and...",Desktop,1,INR,+9194951XXXXXX,Gift for colleague,theodoreXXX@XXXXX.com
29,54fb2e67-b087-4522-a5c4-af69f95cd9df,Jalsa Menon,Male,24,Chhattisgarh,Durg,Durg Branch,Savings,b33c79ab-4d81-4d12-baf3-2b13cf67608a,28-01-2025,...,Electronics,60905.65,ATM Booth Kiosk,"Durg, Chhattisgarh",POS,1,INR,+9195352XXXXXX,Mobile phone payment,jalsaXXX@XXXXXXX.com
32,ad5f8799-179f-4b1d-887c-cace22837d3e,Ishanvi Kant,Female,39,Uttar Pradesh,Varanasi,Varanasi Branch,Business,0d970d8f-dd99-401b-a7ba-83a276a7cf49,19-01-2025,...,Groceries,88389.09,QR Code Scanner,"Varanasi, Uttar Pradesh",Mobile,1,INR,+9199750XXXXXX,Property tax payment,ishanviXXXXX@XXXXX.com
54,6e9eaae0-ed2c-40b7-af0c-14e02647a39c,Peter Dhawan,Female,52,Manipur,Imphal,Imphal Branch,Checking,402d686c-e7b1-4857-b8b0-b6c384b56611,10-01-2025,...,Restaurant,56382.68,Wearable Device,"Imphal, Manipur",POS,1,INR,+9191682XXXXXX,Grocery delivery,peterXXXXX@XXXXXX.com
76,f7d6bcdb-2c7e-4456-9cbc-1516a9186b55,Gopal Ratta,Male,28,Haryana,Gurugram,Gurugram Branch,Savings,f2f00aa5-0896-4cc1-bb52-0f7822b1e741,23-01-2025,...,Groceries,75460.68,ATM,"Gurugram, Haryana",Mobile,1,INR,+9198024XXXXXX,Bank transfer,gopalXXXX@XXXXXXX.com


In [35]:
df_sel_cols = df[["Transaction_Amount","Merchant_Category","Transaction_Device","Transaction_Description","Is_Fraud"]]


In [36]:
df_sel_df = df_sel_cols[df_sel_cols["Is_Fraud"]==1]

In [37]:
df_sel_df

,Transaction_Amount,Merchant_Category,Transaction_Device,Transaction_Description,Is_Fraud
17,85269.30,Restaurant,Biometric Scanner,Gift for colleague,1
29,70219.41,Electronics,ATM Booth Kiosk,Mobile phone payment,1
32,28020.64,Groceries,QR Code Scanner,Property tax payment,1
54,37118.69,Restaurant,Wearable Device,Grocery delivery,1
76,17317.33,Groceries,ATM,Bank transfer,1
...,...,...,...,...,...
199883,52617.13,Groceries,POS Mobile App,POS transaction,1
199928,80190.03,Health,Biometric Scanner,Subscription payment,1
199930,19133.61,Groceries,Voice Assistant,Mobile phone payment,1
199985,12258.02,Electronics,ATM Booth Kiosk,Membership subscription,1


In [38]:
df_sel_df.columns

Index(['Transaction_Amount', 'Merchant_Category', 'Transaction_Device',
       'Transaction_Description', 'Is_Fraud'],
      dtype='object')

In [39]:
df_sel_df.head()

,Transaction_Amount,Merchant_Category,Transaction_Device,Transaction_Description,Is_Fraud
17,85269.30,Restaurant,Biometric Scanner,Gift for colleague,1
29,70219.41,Electronics,ATM Booth Kiosk,Mobile phone payment,1
32,28020.64,Groceries,QR Code Scanner,Property tax payment,1
54,37118.69,Restaurant,Wearable Device,Grocery delivery,1
76,17317.33,Groceries,ATM,Bank transfer,1


In [40]:
df_final = df_sel_cols

In [41]:
df_final.shape

(200000, 5)

In [42]:
df_final["Context_Text"] = df_final.apply(
    lambda row: f"The transaction was at a {row['Merchant_Category']} using a {row['Transaction_Device']}, and the description was {row['Transaction_Description']}.",
    axis=1
)

C:\Users\vinay\AppData\Local\Temp/ipykernel_8532/3946729481.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["Context_Text"] = df_final.apply(


In [43]:
df_final.head()

,Transaction_Amount,Merchant_Category,Transaction_Device,Transaction_Description,Is_Fraud,Context_Text
0,32415.45,Restaurant,Voice Assistant,Bitcoin transaction,0,The transaction was at a Restaurant using a Vo...
1,43622.60,Restaurant,POS Mobile Device,Grocery delivery,0,The transaction was at a Restaurant using a PO...
2,63062.56,Groceries,ATM,Mutual fund investment,0,The transaction was at a Groceries using a ATM...
3,14000.72,Entertainment,POS Mobile App,Food delivery,0,The transaction was at a Entertainment using a...
4,18335.16,Entertainment,Virtual Card,Debt repayment,0,The transaction was at a Entertainment using a...


In [44]:
df_final.drop(columns=["Merchant_Category","Transaction_Device","Transaction_Description"],inplace=True)

C:\Users\vinay\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [45]:
df_final.head()

,Transaction_Amount,Is_Fraud,Context_Text
0,32415.45,0,The transaction was at a Restaurant using a Vo...
1,43622.60,0,The transaction was at a Restaurant using a PO...
2,63062.56,0,The transaction was at a Groceries using a ATM...
3,14000.72,0,The transaction was at a Entertainment using a...
4,18335.16,0,The transaction was at a Entertainment using a...


In [ ]:
## Embedding with Alminilml6v2

In [51]:
# Load MiniLM model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate embeddings
df_final["Embeddings"] = df_final["Context_Text"].apply(lambda x: model.encode(x))


KeyboardInterrupt: 

In [ ]:
# Save with pickle (best for Python objects like numpy arrays)
df_final.to_pickle("saving_df_embeddings.pkl")

# Later load it back without recomputing
# df_final = pd.read_pickle("saving_df_embeddings.pkl")


In [ ]:
# Convert embeddings into array
X_text = np.vstack(df_final["Embeddings"].values)

# Scale transaction amount

scaler = StandardScaler()
amount_scaled = scaler.fit_transform(df[["Transaction_Amount"]])

# Final feature set: embeddings + amount
X = np.hstack([X_text, amount_scaled])

# Target
y = df_final["Is_Fraud"].values


In [ ]:
## Training

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
X_train.shape,y_train.shape 

In [ ]:
X_test.shape, y_test.shape

In [ ]:
clf = RandomForestClassifier(n_estimators=200, class_weight="balanced", random_state=42)
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)[:,1]

In [ ]:
print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))